# 나이 그룹 별 화재 건수

In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import warnings

# 경고 무시
warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Pretendard'
plt.rcParams['axes.unicode_minus'] = False

# 데이터 로드
fire_data = pd.read_csv('./filtered_data/gu_fire_rate_per10k.csv', encoding='utf-8')
age_data = pd.read_csv('./filtered_data/group_population_ratio_byGu.csv', encoding='utf-8')

print("화재율 데이터 (gu_fire_rate_per10k.csv):")
print(fire_data.head())
print("\n나이 그룹 데이터 (group_population_ratio_byGu.csv):")
print(age_data.head())

화재율 데이터 (gu_fire_rate_per10k.csv):
   자치구            인구수  화재율_1만명당
0  강남구  655545.114288  6.437391
1  강동구  509888.837375  3.353672
2  강북구  287913.541725  6.946530
3  강서구  526891.403000  5.200313
4  관악구  450404.012013  6.238843

나이 그룹 데이터 (group_population_ratio_byGu.csv):
   자치구  미성년자_비율  청년_비율  중년_비율  장년_비율  고령_비율
0  종로구    12.61  33.51  31.61  11.04  11.23
1   중구    11.26  35.61  31.63  10.68  10.83
2  용산구    12.53  36.21  31.43   9.94   9.89
3  성동구    13.87  33.73  31.88  10.60   9.92
4  광진구    14.54  35.73  30.08  10.27   9.38


In [10]:
import matplotlib.pyplot as plt
import os
from scipy import stats

# 데이터 병합: gu_fire_rate_per10k와 group_population_ratio_byGu
merged_data = fire_data.merge(age_data, left_on='자치구', right_on='자치구', how='inner')

print("병합된 데이터:")
print(merged_data.head())
print(f"\n병합된 행 수: {len(merged_data)}")
print(f"컬럼: {merged_data.columns.tolist()}")

# results 디렉터리 확인/생성
os.makedirs('./results', exist_ok=True)

# 나이대별 그래프 생성 및 저장
age_groups = ['미성년자_비율', '청년_비율', '중년_비율', '장년_비율', '고령_비율']

for age_group in age_groups:
    fig, ax = plt.subplots(figsize=(11, 7))
    
    # 산점도 생성
    ax.scatter(merged_data[age_group], merged_data['화재율_1만명당'], alpha=0.6, s=120, color='#2E86AB', edgecolors='black', linewidth=1.2)
    
    # 상관계수 및 p-value 계산
    correlation, p_value = stats.pearsonr(merged_data[age_group], merged_data['화재율_1만명당'])
    
    # 회귀선 추가
    z = np.polyfit(merged_data[age_group], merged_data['화재율_1만명당'], 1)
    p_poly = np.poly1d(z)
    x_sorted = merged_data[age_group].sort_values()
    ax.plot(x_sorted, p_poly(x_sorted), "r-", alpha=0.8, linewidth=2.5, label=f'Regression line (slope={z[0]:.3f})')
    
    # 회귀계수 및 p-value를 텍스트 박스로 표시
    textstr = f'상관계수: {correlation:.4f}\np-value: {p_value:.6f}\nR²: {correlation**2:.4f}'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.85)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=11, verticalalignment='top', bbox=props, fontweight='bold')
    
    # 제목 및 라벨
    ax.set_title(f'{age_group} 인구비율 vs 화재율(1만명당)', fontsize=14, fontweight='bold', pad=15)
    ax.set_xlabel(f'{age_group} (%)', fontsize=12, fontweight='bold')
    ax.set_ylabel('화재율 (1만명당)', fontsize=12, fontweight='bold')
    ax.legend(fontsize=10, loc='upper right')
    ax.grid(True, alpha=0.3, linestyle='--')
    
    # 각 점에 구명 라벨 추가
    for idx, row in merged_data.iterrows():
        ax.annotate(row['자치구'], (row[age_group], row['화재율_1만명당']), 
                   fontsize=8.5, alpha=0.8, xytext=(4, 4), textcoords='offset points',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.4))
    
    # 그래프 저장
    plt.tight_layout()
    plt.savefig(f'./results/{age_group}_vs_화재율(1만명당).png', dpi=300, bbox_inches='tight')
    print(f"✓ {age_group} 그래프 저장 완료: ./results/{age_group}_vs_화재율(1만명당).png")
    plt.close()

# 전체 상관계수 및 p-value 요약
print("\n" + "="*60)
print("=== 나이대별 화재율(1만명당) 상관계수 및 p-value ===")
print("="*60)
correlation_summary = {}
pvalue_summary = {}
slope_summary = {}

for age_group in age_groups:
    correlation, p_value = stats.pearsonr(merged_data[age_group], merged_data['화재율_1만명당'])
    z = np.polyfit(merged_data[age_group], merged_data['화재율_1만명당'], 1)
    slope = z[0]
    
    correlation_summary[age_group] = correlation
    pvalue_summary[age_group] = p_value
    slope_summary[age_group] = slope
    
    print(f"\n{age_group}:")
    print(f"  상관계수: {correlation:.6f}")
    print(f"  p-value: {p_value:.8f}")
    print(f"  회귀계수(기울기): {slope:.6f}")
    print(f"  R²: {correlation**2:.6f}")

# 상관계수 및 p-value를 CSV로 저장
result_df = pd.DataFrame({
    'age_group': age_groups,
    'correlation': [correlation_summary[ag] for ag in age_groups],
    'p_value': [pvalue_summary[ag] for ag in age_groups],
    'slope': [slope_summary[ag] for ag in age_groups],
    'r_squared': [correlation_summary[ag]**2 for ag in age_groups]
})
result_df.to_csv('./results/age_fire_rate_correlation.csv', index=False, encoding='utf-8')
print("\n✓ 상관계수, p-value, 회귀계수 요약 저장: ./results/age_fire_rate_correlation.csv")
print("\n" + "="*60)

병합된 데이터:
   자치구            인구수  화재율_1만명당  미성년자_비율  청년_비율  중년_비율  장년_비율  고령_비율
0  강남구  655545.114288  6.437391    19.93  29.12  33.69   8.29   8.98
1  강동구  509888.837375  3.353672    17.63  26.53  32.28  11.82  11.73
2  강북구  287913.541725  6.946530    13.80  27.58  31.81  12.61  14.21
3  강서구  526891.403000  5.200313    15.99  31.69  30.41  10.66  11.25
4  관악구  450404.012013  6.238843    10.99  41.10  28.14   9.59  10.18

병합된 행 수: 25
컬럼: ['자치구', '인구수', '화재율_1만명당', '미성년자_비율', '청년_비율', '중년_비율', '장년_비율', '고령_비율']
✓ 미성년자_비율 그래프 저장 완료: ./results/미성년자_비율_vs_화재율(1만명당).png
✓ 미성년자_비율 그래프 저장 완료: ./results/미성년자_비율_vs_화재율(1만명당).png
✓ 청년_비율 그래프 저장 완료: ./results/청년_비율_vs_화재율(1만명당).png
✓ 청년_비율 그래프 저장 완료: ./results/청년_비율_vs_화재율(1만명당).png
✓ 중년_비율 그래프 저장 완료: ./results/중년_비율_vs_화재율(1만명당).png
✓ 중년_비율 그래프 저장 완료: ./results/중년_비율_vs_화재율(1만명당).png
✓ 장년_비율 그래프 저장 완료: ./results/장년_비율_vs_화재율(1만명당).png
✓ 장년_비율 그래프 저장 완료: ./results/장년_비율_vs_화재율(1만명당).png
✓ 고령_비율 그래프 저장 완료: ./results/고령_비율_vs_화재율(1만명당).png

=== 나이대별 

In [13]:
import matplotlib as mpl
mpl.get_cachedir()

'C:\\Users\\User\\.matplotlib'

In [17]:
mpl.matplotlib_fname()

'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\matplotlib\\mpl-data\\matplotlibrc'